In [13]:
import requests
import os

url = "https://buildingtransparency.org/api/rest-auth/login"

querystring = {"tokenless":"false"}

payload = {
    "username": os.getenv("OPENEPD_USER"),
    "password": os.getenv("OPENEPD_PW")
}
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}

response = requests.post(url, json=payload, headers=headers, params=querystring)

response_data = response.json()

#print(response_data)

auth_key = response.json()['key']

print(auth_key)

lZJeGim1LH9Ou0pqhXwSrlMw2y72UV


In [14]:
headers = {
    "Accept": "application/json",
    "Authorization": "Bearer "+auth_key
}

#ndustry_epd_url = "https://buildingtransparency.org/api/industry_epds"
product_epd_url = "https://buildingtransparency.org/api/epds"

querystring = {"page_size":"250"}

#indus_response = requests.get(industry_epd_url, headers=headers, params=querystring)
prod_response = requests.get(product_epd_url, headers=headers, params=querystring)

#indus_results = indus_response.json()
prod_results = prod_response.json()

In [42]:
#prod_results[93]
lcia = list(prod_results[93]['pcr']['lcia_requirements'].keys())[0]
print(type(lcia))
print(prod_results[48]['open_xpd_uuid'])


<class 'str'>
ec3rp05b


In [43]:
impact_info_url = "https://openepd.buildingtransparency.org/api/epds/"+"ec3rp05b"

response_impact = requests.get(impact_info_url, headers=headers)
resp = response_impact.json()
# print(resp['impacts'][lcia]['gwp']["A1A2A3"]['mean'])
# print(resp['impacts'][lcia]['gwp']['C2'])
# print(resp['impacts'][lcia]['gwp']['C3'])
# print(resp['impacts'][lcia]['gwp']['C4'])
# print(resp['impacts'][lcia]['odp'])

resp['impacts']

{'EF 3.0': {'gwp': {'A1A2A3': {'mean': 133.0,
    'unit': 'kgCO2e',
    'rsd': 0.28879058156387305,
    'dist': 'log-normal'},
   'A1': None,
   'A2': None,
   'A3': None,
   'A4': None,
   'A5': None,
   'B1': None,
   'B1_years': None,
   'B2': None,
   'B2_years': None,
   'B3': None,
   'B3_years': None,
   'B4': None,
   'B4_years': None,
   'B5': None,
   'B5_years': None,
   'B6': None,
   'B6_years': None,
   'B7': None,
   'B7_years': None,
   'C1': None,
   'C2': None,
   'C3': None,
   'C4': None,
   'D': None,
   'C_scenarios': None}}}

In [5]:
#total_pages = int(indus_response.headers['X-Total-Pages'])
total_pages2 = int(prod_response.headers['X-Total-Pages'])

#print('Industry EPD Pages: ', total_pages)
print('Product EPD Pages: ', total_pages2)

Product EPD Pages:  722


In [21]:
product_epds = []

for i in range(0,2):
        querystring = {
                "page_number": i+1,
                "page_size":"250"}
        response = requests.get(product_epd_url, headers=headers, params=querystring)
        epd_data = response.json()
        for epd in epd_data:
                product_epds.append(epd)
        print("successfully completed page "+str(i+1))

successfully completed page 1
successfully completed page 2


In [15]:
EC3_epds = []

for i in range(0,2):
        querystring = {
                "page_number": i+1,
                "page_size":"250"}
        response = requests.get(product_epd_url, headers=headers, params=querystring)
        epd_data = response.json()
        for epd in epd_data:
                EC3_epds.append(epd)
        print("successfully completed page "+str(i+1))

successfully completed page 1
successfully completed page 2


In [22]:
import requests
import time

epd_impacts = []
impact_info_url = "https://openepd.buildingtransparency.org/api/epds/"

for i in range(0, 266):
    xpdID = EC3_epds[i]['open_xpd_uuid']
    
    while True:
        response_impact = requests.get(impact_info_url + xpdID, headers=headers)
        if response_impact.status_code == 200:
            epd_impacts.append(response_impact.json())
            break
        elif response_impact.status_code == 429:  # Throttling status code
            error_details = response_impact.json()
            wait_time = int(error_details.get('detail', '').split()[-2])
            print(f"Throttled on json {i}, Waiting for {wait_time} seconds")
            time.sleep(wait_time)
        else:
            print(f"Error: {response_impact.status_code}")
            break
    
    if i % 25 == 0:
        print(f'done with impact {i}')


done with impact 0
Throttled on json 19, Waiting for 2 seconds
Throttled on json 20, Waiting for 2 seconds
Throttled on json 21, Waiting for 3 seconds
Throttled on json 22, Waiting for 2 seconds
Throttled on json 23, Waiting for 3 seconds
Throttled on json 24, Waiting for 3 seconds
Throttled on json 25, Waiting for 3 seconds
done with impact 25
Throttled on json 26, Waiting for 3 seconds
Throttled on json 27, Waiting for 3 seconds
Throttled on json 28, Waiting for 2 seconds
Throttled on json 29, Waiting for 3 seconds
Throttled on json 30, Waiting for 2 seconds
Throttled on json 31, Waiting for 3 seconds
Throttled on json 32, Waiting for 3 seconds


In [21]:
epd_impacts[34]

{'id': 'ec3n0zrr',
 'ref': 'https://openepd.buildingtransparency.org/api/epds/EC3N0ZRR',
 'ec3': {'uaGWP_a1a2a3_traci21': 337.66724496804227,
  'uaGWP_a1a2a3_ar5': 340.7293473406328,
  'category': 'Concrete >> ReadyMix',
  'manufacturer_specific': True,
  'plant_specific': True,
  'product_specific': True,
  'batch_specific': None,
  'supply_chain_specificity': 0.85,
  'original_data_format': 'custom_api'},
 'openepd_version': '0.1',
 'doctype': 'OpenEPD',
 'version': None,
 'language': 'en',
 'private': False,
 'declaration_url': 'https://api.climateearth.com/api/mixes/epd/A66656D6-15D1-45B3-9996-6A088905F03D/true',
 'lca_discussion': '\n\n## Product Component Notes\ncrushed aggregate (ASTM C33), natural aggregate (ASTM C33), Portland cement (ASTM C150), fly ash (ASTM C618), batch water (ASTM C1602), admixture (ASTM C494)\n\n## Source Data Notes\nAggregate (crushed): US-EI (2016): “Gravel, crushed, at mine/US”, 2001 Aggregate: Supplier specific primary data, 2016 Aggregate: Supplier s

In [12]:
type(epd_impacts[38])

dict

In [ ]:
impact_data = {

        f"{lcia}": {
            "gwp_total": {
            "A1_A3": epd_impacts[17]['impacts'][lcia]['gwp']["A1A2A3"]['mean'],
            "C2": epd_impacts[0]['impacts'][lcia]['gwp']['C2'],
            "C3": epd_impacts[0]['impacts'][lcia]['gwp']['C3'],
            "C4": epd_impacts[0]['impacts'][lcia]['gwp']['C4'],
            "D1": epd_impacts[0]['impacts'][lcia]['gwp']['D']
            },
        }
}

print(impact_data)

In [7]:
def string_split(s):
    parts = s.split(' ', 1)
    
    magnitude = float(parts[0])
    unit = parts[1]
    
    return magnitude, unit


In [8]:
from datetime import datetime
import requests
import json

#category = ['GWP-total','GWP-biogenic','GWP-fossil','GWP-IOBC']

headers = {
    "Accept": "application/json",
    "Authorization": "Bearer "+auth_key
}

product_epd_url = "https://buildingtransparency.org/api/epds"
impact_info_url = "https://openepd.buildingtransparency.org/api/epds/"

EPDs = []
#EC3_epds = []
product_epds = []
error_epds = []
none = "Unmapped"
page = 0

for x in range(0,225):
    # if x % 250 == 0 or page == 0:
    #     page += 1
    #     EC3_epds.extend(get_page(page))


    date_validity_ends_str = EC3_epds[x]['date_validity_ends']
    #date_validity_ends = datetime.strptime(date_validity_ends_str, '%Y-%m-%d')

    #if date_validity_ends < datetime.now():
        #print(f"EPD {x} expired, skipping...")
        #continue

    # xpdID = EC3_epds[x]['open_xpd_uuid']
    # response_impact = requests.get(impact_info_url+xpdID, headers=headers)
    # impact = response_impact.json()

    material_data = {} 

    ############################### DATA PATHS #################################
    #lcia = list(EC3_epds[x]['pcr']['lcia_requirements'].keys())[0]

    try:
        lcia_requirements = EC3_epds[x]['pcr']['lcia_requirements']
        if lcia_requirements is None:
            raise ValueError("lcia_requirements is None")

        lcia = list(lcia_requirements.keys())[0]
    except (AttributeError, ValueError) as e:
        print(f"Skipping index {x} due to missing or invalid lcia_requirements: {e}")
        error_epds.append(f'json {x} lcia error')
        continue

    transport_dist, transport_unit = string_split(EC3_epds[x]['category']['default_distance'])
    dens, dens_unit = string_split(EC3_epds[x]['density'])
    mass, mass_unit = string_split(EC3_epds[x]['category']['mass_per_declared_unit'])
    

    material_data['material_id'] = f'EC3_{EC3_epds[x]['category']['id']}'
    material_data.update({'validity':{'gte':EC3_epds[x]['category']['created_on'], 'lte':date_validity_ends_str}})
    material_data.update({'display_name':EC3_epds[x]['category']['display_name']})
    material_data.update({"source": "EC3 Product Specific EPDs"})
    material_data.update({'declared_unit':EC3_epds[x]['category']['declared_unit']})
    material_data.update({'A4_transportation':{'mode':EC3_epds[x]['category']['default_transport_mode'], 'distance':{'qty':transport_dist, 'unit':transport_unit}}})
    material_data.update({'service_life':EC3_epds[x]['reference_service_life']})
    material_data.update({'waste_rate':none})
    material_data.update({'density':{'qty':dens, 'unit':dens_unit}})
    material_data.update({'item_mass':{'qty':mass, 'unit':mass_unit}})


    # try:
    #     material_data.update({'impacts':{lcia:{'gwp_total':{'A1_A3':epd_impacts['impacts'][lcia]['gwp']["A1A2A3"]['mean'], 'C2':epd_impacts['impacts'][lcia]['gwp']['C2'], 'C3':epd_impacts['impacts'][lcia]['gwp']['C3'], 'C4':epd_impacts['impacts'][lcia]['gwp']['C4'], 'D':epd_impacts['impacts'][lcia]['gwp']['D']}}}})
    # except KeyError:
        # print(f'Key error on json #{x}')
        # message = f'json {x} key error'
        # error_epds.append(message)
        # material_data.update({'characteristics':message})




    desc = EC3_epds[x]['category']['description']
    pcr_name = EC3_epds[x]['pcr']['name']
    pcr_id = EC3_epds[x]['pcr']['id']
    short_link = EC3_epds[x]['pcr']['short_link']
    created_on = EC3_epds[x]['category']['created_on']

    thckqty = EC3_epds[x]['thickness']
    thckU = EC3_epds[x]['thickness_per_declared_unit']


    dens = EC3_epds[x]['density']
    sdens = EC3_epds[x]['specific_density']

    #try:
    impact_data = {

        f"{lcia}": {
            "gwp_total": {
            "A1_A3": epd_impacts[x]['impacts'][lcia]['gwp']["A1A2A3"]['mean'],
            "C2": epd_impacts[x]['impacts'][lcia]['gwp']['C2'],
            "C3": epd_impacts[x]['impacts'][lcia]['gwp']['C3'],
            "C4": epd_impacts[x]['impacts'][lcia]['gwp']['C4'],
            "D1": epd_impacts[x]['impacts'][lcia]['gwp']['D']
            },
            # "gwp_fossil": {
            # "A1_A3": 2.0,
            # "C2": 0.1,
            # "C3": 0.04,
            # "C4": 0.01,
            # "D1": -0.48
            # }
        },
        "characteristics": {}
    }
        
    material_data['impacts'] = impact_data
    # except KeyError:
    #     print(f'Key error on json #{x}')
    #     message = f'json {x} key error'
    #     error_epds.append(message)
    #     material_data.update({'characteristics':message})
    
    EPDs.append(material_data)
    print(f'json #{x} is done')

with open('epds.json', 'w') as json_file:
    json.dump(EPDs, json_file, indent=4)

print("All data saved to epds.json")

with open('error_epds.json', 'w') as json_file:
    json.dump(error_epds, json_file, indent=4)

print("All error data saved to epds.json")


json #0 is done
json #1 is done
json #2 is done
json #3 is done
json #4 is done
json #5 is done
json #6 is done
json #7 is done
json #8 is done
json #9 is done
json #10 is done
json #11 is done
json #12 is done
json #13 is done
json #14 is done
json #15 is done
json #16 is done
json #17 is done
json #18 is done
json #19 is done
json #20 is done
json #21 is done
json #22 is done
json #23 is done
json #24 is done
json #25 is done
json #26 is done
json #27 is done
json #28 is done
json #29 is done
json #30 is done
json #31 is done
json #32 is done
json #33 is done
json #34 is done
json #35 is done
json #36 is done


KeyError: 'impacts'

In [6]:
EPDs

[{'material_id': 'b03dba1dca5b49acb1a5aa4daab546b4',
  'validity': {'gte': '2018-06-26T01:11:48.938840Z', 'lte': '2021-10-03'},
  'display_name': 'Ready Mix',
  'source': 'Product Specific EPDs',
  'declared_unit': '1 m3',
  'A4_transportation': {'mode': 'concrete mixer truck',
   'distance': {'qty': '51.49888 km', 'unit': 'km (plchlder)'}},
  'service_life': None,
  'waste_rate': 'Unmapped',
  'density': {'qty': '2400 kg / m^3', 'unit': 'kg/m3 (plchlder)'},
  'item_mass': {'qty': '2400 kg', 'unit': 'kg (plchlder)'},
  'impacts': {'TRACI 2.1': {'gwp_total': {'A1_A3': 339.0,
     'C2': None,
     'C3': None,
     'C4': None,
     'D': None}}}}]

In [12]:
#epd_data[0]['open_xpd_uuid']
len(product_epd_ids)
product_epd_ids[0]



'ec34u6kx'

In [7]:
import requests
import json
#ec3b6w55
#ec36urza
impact_info_url = "https://openepd.buildingtransparency.org/api/epds/"+"ec3b6w55"

response_impact = requests.get(impact_info_url, headers=headers)
resp = response_impact.json()

file_path = 'impacts2.json'

with open(file_path, 'w') as json_file:
    json.dump(resp, json_file, indent=4)


In [ ]:
    material_data = {
        "material_id": 'to be assigned later',
        "display_name": EPDname,
        "source": none,
        "description": f'{desc},',
        #"validity": date_validity_ends_str,
        "reference_pcr": {
            "name": pcr_name,
            "id": pcr_id,
            "link": short_link
        },
        "thickness": {
            "qty": thckqty,
            "unit": thckU
        },
        "service_life": seLi,
        "waste_rate": none,
        "transportation": {
            "mode": tMode,
            "distance": dist
        },
        "density": {
            "qty": dens,
            "unit": none
        },
        "linear_density": {
            "qty": sdens,
            "unit": none
        },
        "declared_unit": unit,
        "lcia_method": lcia,
        "background_database": none,
        "impacts": {
            "gwp_total": {
                #"A1": A11, "A2": A12, "A3": A13, "A4": A14, "A5": A15, "C1": C11,
                #"C2": C12, "C3": C13, "C4": C14, "D": D1, "D1": D11, "D2": D12
            },
            "gwp_biogenic": {
                #"A1": A21, "A2": A22, "A3": A23, "A4": A24, "A5": A25, "C1": C21,
                #"C2": C22, "C3": C23, "C4": C24, "D": D2, "D1": D21, "D2": D22
            },
            "gwp_fossil": {
                #"A1": A31, "A2": A32, "A3": A33, "A4": A34, "A5": A35, "C1": C31,
                #"C2": C32, "C3": C33, "C4": C34, "D": D3, "D1": D31, "D2": D32
            },
            "gwp_iobc": {
                #"A1": A41, "A2": A42, "A3": A43, "A4": A44, "A5": A45, "C1": C41,
                #"C2": C42, "C3": C43, "C4": C44, "D": D4, "D1": D41, "D2": D42
            }
        },
        "characteristics": {}
    },